# Experimenting Notebook

This notebook aims develop and test the code that will be used on the app.

In [3]:
import nltk
import re
import string
from nltk.tokenize import sent_tokenize as nlkt_sent_tokenize
from nltk.tokenize import word_tokenize as nlkt_word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords
import numpy as np
from scipy.spatial.distance import cosine
from newspaper import Article